In [6]:
from langchain_community.llms import Ollama
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_ollama import OllamaLLM
from langchain_ollama import ChatOllama

In [11]:
#Cargamos la IA Ollama con langchain
from langchain_ollama import ChatOllama

# Configuración correcta usando ChatOllama
# Asegúrate del nombre del modelo y su compatibilidad.
model_name = "llama3"  # Verifica si este modelo es compatible

try:
    llm = ChatOllama(model=model_name)
    respuesta = llm.invoke("Hi, What do you need?")
    print(respuesta)
except Exception as e:
    print(f"Error al invocar el modelo: {e}")


content="Hello! I'm happy to chat with you. As a conversational AI, my purpose is to understand and respond to your questions, thoughts, or topics you'd like to discuss. I don't have any specific needs per se, but I can help with:\n\n1. Answering questions: If you're stuck on something, feel free to ask me anything, from science and history to language and culture.\n2. Brainstorming ideas: Want to generate some creative ideas or explore different perspectives? I'm here to help!\n3. Language practice: If you're learning a new language or want to improve your writing skills, we can have a conversation or exchange texts.\n4. Emotional support: Sometimes all we need is someone to listen. I'm here for that too!\n\nWhat's on your mind today? Do you have a specific topic or question in mind, or would you like me to suggest some conversation starters?" additional_kwargs={} response_metadata={'model': 'llama3', 'created_at': '2025-02-22T18:58:00.481977Z', 'done': True, 'done_reason': 'stop', 't

In [12]:
loader = PyMuPDFLoader("010-25.pdf")

In [13]:
data_pdf = loader.load()

In [ ]:
#cada posicion en un array es una pagina del documento, la [0] es la pag 1
data_pdf[0]

Document(metadata={'producer': 'Skia/PDF m66', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36', 'creationdate': '2018-05-28T13:10:38+00:00', 'source': '010-25.pdf', 'file_path': '010-25.pdf', 'total_pages': 13, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2018-05-28T13:10:38+00:00', 'trapped': '', 'modDate': "D:20180528131038+00'00'", 'creationDate': "D:20180528131038+00'00'", 'page': 0}, page_content='5/28/2018\nInsurance 2030--The impact of AI on the future of insurance | McKinsey & Company\nhttps://www.mckinsey.com/industries/financial-services/our-insights/insurance-2030-the-impact-of-ai-on-the-future-of-insurance\n1/13\nArticle\nApril 2018\nInsurance 2030—The impact of AI\non the future of insurance\nBy Ramnath Balasubramanian, Ari Libarikian, and Doug McElhaney\nW\nThe industry is on the verge of a seismic, tech-driven shift. A focus\non four areas can po

In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=500)
docs = text_splitter.split_documents(data_pdf)

In [22]:
#Validamos cada chunk del arreglo que se esta haciendo
docs[3]

Document(metadata={'producer': 'Skia/PDF m66', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36', 'creationdate': '2018-05-28T13:10:38+00:00', 'source': '010-25.pdf', 'file_path': '010-25.pdf', 'total_pages': 13, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2018-05-28T13:10:38+00:00', 'trapped': '', 'modDate': "D:20180528131038+00'00'", 'creationDate': "D:20180528131038+00'00'", 'page': 2}, page_content='5/28/2018\nInsurance 2030--The impact of AI on the future of insurance | McKinsey & Company\nhttps://www.mckinsey.com/industries/financial-services/our-insights/insurance-2030-the-impact-of-ai-on-the-future-of-insurance\n3/13\nExhibit 1\nAs AI becomes more deeply integrated in the industry, carriers must position\nthemselves to respond to the changing business landscape. Insurance executives must\nunderstand the factors that will contribute to this change and how

In [23]:
docs[4]

Document(metadata={'producer': 'Skia/PDF m66', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36', 'creationdate': '2018-05-28T13:10:38+00:00', 'source': '010-25.pdf', 'file_path': '010-25.pdf', 'total_pages': 13, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2018-05-28T13:10:38+00:00', 'trapped': '', 'modDate': "D:20180528131038+00'00'", 'creationDate': "D:20180528131038+00'00'", 'page': 3}, page_content='5/28/2018\nInsurance 2030--The impact of AI on the future of insurance | McKinsey & Company\nhttps://www.mckinsey.com/industries/financial-services/our-insights/insurance-2030-the-impact-of-ai-on-the-future-of-insurance\n4/13\ndecade.\nExplosion of data from connected devices\nIn industrial settings, equipment with sensors have been omnipresent for some time,\nbut the coming years will see a huge increase in the number of connected consumer\ndevices. The penetrat

In [ ]:
#Realizamos los embedings
#%pip install fastembed
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
embed_model = FastEmbedEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
# Install chromadb package
#%pip install chromadb

from langchain_community.vectorstores import Chroma
vs = Chroma.from_documents(
    documents=docs,
    embedding=embed_model,
    persist_directory="chroma_db_dir",
    collection_name="prueba"
)

In [33]:
vectorstore = Chroma(
                    embedding_function=embed_model,
                    persist_directory="chroma_db_dir",
                    collection_name="prueba")
retriever = vectorstore.as_retriever(search_kwargs={'k':3})

C:\Users\pipe-\AppData\Local\Temp\ipykernel_9812\1666141679.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [38]:
from langchain.prompts import PromptTemplate

custom_prompt_template = """Usa la siguinete información para responder las preguntas que te hago
Si no sabes, di que no lo sabes y no inventes respuesta
                            
Contexto: {context}
Pregunta: {question}

Solo devuelve la respuesta útil a continuación y nada más
Respuesta útil:"""
prompt = PromptTemplate(template=custom_prompt_template,
                        input_variables=['context', 'question'])

In [39]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm = llm,
                                chain_type = "stuff",
                                retriever = retriever,
                                return_source_documents = True,
                                chain_type_kwargs = {"prompt": prompt})


In [45]:
response = qa.invoke({"query": "who is the author?"})

In [47]:
response['result']

'La autoría es compartida entre Ramnath Balasubramanian, Ari Libarikian, y Doug McElhaney.'